In [1]:
from django.db import connection, reset_queries
import sqlparse
reset_queries()
def print_queries():
    print("")
    print("----SQL Queries----")
    for q in connection.queries:
        print(sqlparse.format(q["sql"]))

In [2]:
from books.models import Book, Author, Genre

In [3]:
Author.objects.all().delete()
Genre.objects.all().delete()
Book.objects.all().delete()
terry_pratchett = Author.objects.create(name="Terry Pratchett", age=70)
niel_gaiman = Author.objects.create(name="Niel Gaiman", age=57)

fantasy = Genre.objects.create(name="Fantasy")

good_omens = Book.objects.create(title="Good Omens", genre=fantasy)
good_omens.authors.add(terry_pratchett, niel_gaiman)


# Django ORM
![](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

## What is an ORM (Object Relation Manager)?


# A bridge between Python and SQL

![avatar](http://stuffpoint.com/avatar-the-last-airbender/image/59835-avatar-the-last-airbender-avatar-state.jpg)

# Benefits of using a ORM

![](https://i.imgflip.com/rtcwz.jpg)

# Less error prone than writing SQL by hand

# More likely to produce more efficient SQL

# Models
```py
class Author(models.Model):
    name = models.CharField(max_length=255)
    age = models.IntegerField()


class Genre(models.Model):
    name = models.CharField(max_length=255)


class Book(models.Model):
    title = models.CharField(max_length=255)
    authors = models.ManyToManyField(Author, related_name='books')
    genre = models.ForeignKey(Genre, related_name='books',
                              on_delete=models.SET_NULL, null=True)

```

# QuerySets

In [4]:
queryset = Book.objects.filter(title__startswith="G").exclude(genre__name__startswith="B")

for x in queryset:
    print(x.title)

Good Omens


- Lazy

- Chainable

- Cachable

![](https://media.giphy.com/media/AtRaEatCSjC0w/giphy.gif)

# Creating a QuerySet

```py
queryset = Author.objects
```

# Creating instances

In [5]:
douglas_adams = Author.objects.create(name="Douglas Adams", age=42)

# Getting a single instance

In [6]:
Author.objects.get(name="Terry Pratchett")

<Author: Author object (50)>

In [7]:
Author.objects.first()

<Author: Author object (50)>

# Getting multiple instances

In [8]:
Book.objects.all()

<QuerySet [<Book: Book object (27)>]>

In [9]:
Book.objects.filter(title="Good Omens")

<QuerySet [<Book: Book object (27)>]>

In [10]:
Author.objects.filter(age__gt=50)

<QuerySet [<Author: Author object (50)>, <Author: Author object (51)>]>

In [11]:
Book.objects.exclude(title__startswith="Good")

<QuerySet []>

# Modifing existing instances

In [12]:
douglas_adams = Author.objects.get(name="Douglas Adams")
douglas_adams.age = 70
douglas_adams.save()

In [13]:
num_updated = Author.objects.filter(name="Douglas Adams").update(age=70)

# Deleting instances

In [14]:
_ = douglas_adams.delete()

# or Author.objects.filter(name="Douglas Adams").delete()

# Counting instances

In [15]:
len(Book.objects.all())

1

In [16]:
Book.objects.count()

1

In [17]:
Book.objects.exists()

True

# Ordering instances

In [18]:
Author.objects.all().order_by('age')

<QuerySet [<Author: Author object (51)>, <Author: Author object (50)>]>

In [19]:
Author.objects.all().order_by('-age')

<QuerySet [<Author: Author object (50)>, <Author: Author object (51)>]>

# Limiting number or returned instances

In [20]:
Author.objects.all()[5:10]

<QuerySet []>

# Dealing with related models

In [21]:
douglas_adams = Author.objects.create(name="Douglas Adams", age=42)
book = Book.objects.create(title="Hitchhiker's Guild to the Galaxy", genre=fantasy)

book.authors.add(douglas_adams)
book.authors.all()

<QuerySet [<Author: Author object (53)>]>

In [22]:
Book.objects.filter(authors__age__gte=50)

<QuerySet [<Book: Book object (27)>, <Book: Book object (27)>]>

# Tips, Tricks and Gotchas

![](https://i.imgflip.com/h9dtu.jpg)

In [23]:
reset_queries()

books = Book.objects.all()
for b in books:
    print(b.title, b.genre.id)
    
print_queries()

Good Omens 16
Hitchhiker's Guild to the Galaxy 16

----SQL Queries----
SELECT "books_book"."id", "books_book"."title", "books_book"."genre_id" FROM "books_book"
SELECT "books_genre"."id", "books_genre"."name" FROM "books_genre" WHERE "books_genre"."id" = 16
SELECT "books_genre"."id", "books_genre"."name" FROM "books_genre" WHERE "books_genre"."id" = 16


In [24]:
reset_queries()

books = Book.objects.all()
for b in books:
    print(b.title, b.genre_id)
    
print_queries()

Good Omens 16
Hitchhiker's Guild to the Galaxy 16

----SQL Queries----
SELECT "books_book"."id", "books_book"."title", "books_book"."genre_id" FROM "books_book"


In [25]:
reset_queries()

books = Book.objects.all()
for b in books:
    print(b.title, b.genre.name)
    
print_queries()

Good Omens Fantasy
Hitchhiker's Guild to the Galaxy Fantasy

----SQL Queries----
SELECT "books_book"."id", "books_book"."title", "books_book"."genre_id" FROM "books_book"
SELECT "books_genre"."id", "books_genre"."name" FROM "books_genre" WHERE "books_genre"."id" = 16
SELECT "books_genre"."id", "books_genre"."name" FROM "books_genre" WHERE "books_genre"."id" = 16


In [26]:
reset_queries()

books = Book.objects.all()
books = books.select_related('genre')
for b in books:
    print(b.title, b.genre.name)
    
print_queries()

Good Omens Fantasy
Hitchhiker's Guild to the Galaxy Fantasy

----SQL Queries----
SELECT "books_book"."id", "books_book"."title", "books_book"."genre_id", "books_genre"."id", "books_genre"."name" FROM "books_book" LEFT OUTER JOIN "books_genre" ON ("books_book"."genre_id" = "books_genre"."id")


In [27]:
reset_queries()

books = Book.objects.all()
for b in books:
    for a in b.authors.all():
        print(a, b)
        
print_queries()

Author object (50) Book object (27)
Author object (51) Book object (27)
Author object (53) Book object (28)

----SQL Queries----
SELECT "books_book"."id", "books_book"."title", "books_book"."genre_id" FROM "books_book"
SELECT "books_author"."id", "books_author"."name", "books_author"."age" FROM "books_author" INNER JOIN "books_book_authors" ON ("books_author"."id" = "books_book_authors"."author_id") WHERE "books_book_authors"."book_id" = 27
SELECT "books_author"."id", "books_author"."name", "books_author"."age" FROM "books_author" INNER JOIN "books_book_authors" ON ("books_author"."id" = "books_book_authors"."author_id") WHERE "books_book_authors"."book_id" = 28


In [28]:
reset_queries()

books = Book.objects.all()
books = books.prefetch_related('authors')
for b in books:
    for a in b.authors.all():
        print(a, b)
        
print_queries()

Author object (50) Book object (27)
Author object (51) Book object (27)
Author object (53) Book object (28)

----SQL Queries----
SELECT "books_book"."id", "books_book"."title", "books_book"."genre_id" FROM "books_book"
SELECT ("books_book_authors"."book_id") AS "_prefetch_related_val_book_id", "books_author"."id", "books_author"."name", "books_author"."age" FROM "books_author" INNER JOIN "books_book_authors" ON ("books_author"."id" = "books_book_authors"."author_id") WHERE "books_book_authors"."book_id" IN (27, 28)


In [29]:
reset_queries()

books = Book.objects.all()
if books.exists():
    for b in books:
        print(b.title)
        
print_queries()

Good Omens
Hitchhiker's Guild to the Galaxy

----SQL Queries----
SELECT (1) AS "a" FROM "books_book" LIMIT 1
SELECT "books_book"."id", "books_book"."title", "books_book"."genre_id" FROM "books_book"


In [30]:
reset_queries()

books = Book.objects.all()
if books:
    for b in books:
        print(b.title)
        
print_queries()

Good Omens
Hitchhiker's Guild to the Galaxy

----SQL Queries----
SELECT "books_book"."id", "books_book"."title", "books_book"."genre_id" FROM "books_book"
